In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer


In [2]:
df = pd.read_csv('train.csv',usecols=['Age','Fare','SibSp','Parch','Survived'])
df.sample(5)

,Survived,Age,SibSp,Parch,Fare
371,0,18.0,1,0,6.4958
484,1,25.0,1,0,91.0792
70,0,32.0,0,0,10.5000
324,0,NaN,8,2,69.5500
805,0,31.0,0,0,7.7750


**drop all the rows that have missing values**

In [3]:
df.dropna(inplace=True)

In [4]:
df.head()

,Survived,Age,SibSp,Parch,Fare
0,0,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,1,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,0,35.0,0,0,8.0500


In [5]:
df['family'] = df['SibSp'] + df['Parch']
df.head()

,Survived,Age,SibSp,Parch,Fare,family
0,0,22.0,1,0,7.2500,1
1,1,38.0,1,0,71.2833,1
2,1,26.0,0,0,7.9250,0
3,1,35.0,1,0,53.1000,1
4,0,35.0,0,0,8.0500,0


In [6]:
df.drop(columns=['SibSp','Parch'],inplace=True)

In [7]:
df

,Survived,Age,Fare,family
0,0,22.0,7.2500,1
1,1,38.0,71.2833,1
2,1,26.0,7.9250,0
3,1,35.0,53.1000,1
4,0,35.0,8.0500,0
...,...,...,...,...
885,0,39.0,29.1250,5
886,0,27.0,13.0000,0
887,1,19.0,30.0000,0
889,1,26.0,30.0000,0


In [8]:
X = df.drop(columns=['Survived'])
y = df['Survived']

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [10]:
X_train.head(2)

,Age,Fare,family
328,31.0,20.5250,2
73,26.0,14.4542,1


In [11]:
y_train.head(1)

,Survived
328,1


**Without binarization**

In [12]:
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.6223776223776224

In [14]:
clf = DecisionTreeClassifier()
np.mean(cross_val_score(
    clf,
    X,y,cv=10,scoring='accuracy'
))

np.float64(0.6443270735524257)

**Applying Binarization**

**copy = False, meaning inplace changes, copy=True, meaning a new column is created**

In [15]:
from sklearn.preprocessing import Binarizer

# default threshold = 0.0
# so if the person is travelling alone or
# with one person or more
trf = ColumnTransformer([
    ('bin',Binarizer(copy=False),['family'])
],remainder = 'passthrough')

In [22]:
X_train.head(4)

,Age,Fare,family
328,31.0,20.5250,2
73,26.0,14.4542,1
253,30.0,16.1000,1
719,33.0,7.7750,0


In [20]:
X_train_transformed = trf.fit_transform(X_train)
X_test_transformed = trf.transform(X_test)
X_train_transformed

array([[  1.    ,  31.    ,  20.525 ],
       [  1.    ,  26.    ,  14.4542],
       [  1.    ,  30.    ,  16.1   ],
       ...,
       [  0.    ,  41.    , 134.5   ],
       [  1.    ,  33.    ,  20.525 ],
       [  0.    ,  33.    ,   7.8958]])

In [21]:
pd.DataFrame(X_train_transformed,columns=['family','Age','Fare'])

,family,Age,Fare
0,1.0,31.0,20.5250
1,1.0,26.0,14.4542
2,1.0,30.0,16.1000
3,0.0,33.0,7.7750
4,0.0,25.0,13.0000
...,...,...,...
566,1.0,46.0,61.1750
567,0.0,25.0,13.0000
568,0.0,41.0,134.5000
569,1.0,33.0,20.5250


In [23]:
clf = DecisionTreeClassifier()
clf.fit(X_train_transformed,y_train)
y_pred2 = clf.predict(X_test_transformed)
accuracy_score(y_test,y_pred2)

0.6153846153846154

In [24]:
X_trf = trf.fit_transform(X)
np.mean(cross_val_score(
    DecisionTreeClassifier(),
    X_trf,y,cv=10,scoring='accuracy'
))

np.float64(0.6275821596244132)